In [1]:
# Importing the necessary modules. 
import os
import csv
import requests
import time
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

# BSN calendar pull 

1. Define a list of team numbers and names:

https://bsnpr.com/juegos/juego?cid=29382&r=BS         2022 10 35

In [305]:
# Define a list of team numbers and names
import sys
import traceback

def boxscore_scrape(year, month, day):
    try:
        date_str = f"{year}{str(month).rjust(2, '0')}{str(day).rjust(2, '0')}"
        #f"{year}{month}{str(day).rjust(2, '0')}"
        url = 'https://bsnpr.com/juegos/juego?r=BS' + str(date_str)
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')

        tables = soup.findAll("table", class_="tabla_stats")
        
        if len(tables) < 2:
            print(f"No boxscore found for date {date_str}")
        else:
            t_boxscore = tables[1]
            t_hometeam = tables[2]
            t_awayteam = tables[3]
              
            folder_path = f"/Users/jeanzayas/Desktop/Divergence/Portfolio/Sports Analysis/Basketball/BSN/dataset/{year}"
            if not os.path.exists(folder_path):
                os.makedirs(folder_path)
            
                        # Extract the column names from the table header
            headers = [th.text.strip() for th in t_boxscore.findAll("th")]
            
            print("{}".format(headers))

        
            # extract the time and date of the game... 
            datetime_str = headers[0]
            
            game_date = datetime_str.split()[0] 
            gametime = datetime_str.split()[1] if datetime_str.find(" ") > 0 else "8:00"
            am_pm = datetime_str.split()[-1] if datetime_str.find(" ") > 0 else "PM"
            
            df_fecha = pd.DataFrame({'Day': game_date, 'Time': gametime, 'AM:PM': am_pm}, index=[0])
            
            
            #df_fecha = pd.concat([df_gameday, df_gametime], axis=1)
          
            # extract the team names and scores from the table_boxscore
            team_names = []
            team_scores = []
            for row in t_boxscore.findAll("tr")[1:]:
                cols = row.findAll("td")
                team_names.append(cols[0].text.strip())
                team_scores.append(cols[-1].text.strip())

            # Extract the table data
            table_data = []
            for row in t_boxscore.findAll("tr")[1:]:
                cols = row.findAll("td")[1:-1]
                row_data = [col.text.strip() for col in cols]
                table_data.append(row_data)

            df_table = pd.DataFrame(table_data[:4], columns=headers[1:])
            ## Check if the boxscore has at least 4 quarters
            if len(table_data) > 4:
                # Create a DataFrame from the table data with 4 columns for the first 4 quarters

                # If there are more than 4 quarters, add an 'OT' column for each additional quarter
                for i in range(5, len(table_data)):
                    column_name = f"OT{i-3}"  # Name the column 'OT5' for the 5th quarter, 'OT6' for the 6th, and so on
                    df_table[column_name] = table_data[i]

            # Add the boxscore to the list of errors if it doesn't have at least 4 quarters
            else:
                # Create a DataFrame for the team names and scores 
                df_team = pd.DataFrame({'Team': team_names, 'Total': team_scores})

                # Concatenate the two DataFrames
                df = pd.concat([df_team, df_table], axis=1)
                df = pd.concat([df, df_fecha], axis=1)
                df.drop(2, inplace=True)
                # Extract the team names and scores from the DataFrame
                home_team = df.iloc[0]['Team']
                home_score = df.iloc[0]['Total']
                away_team = df.iloc[1]['Team']
                away_score = df.iloc[1]['Total']

                # Print the final score in the desired format
                print(f"{home_team} {home_score} v {away_team} {away_score}")

                #print(df)

            # Extract the table data fro the home team
            table_data_home = []
            for row in t_hometeam.findAll("tr")[1:]:
                cols = row.findAll("td")
                row_data = [col.text.strip() for col in cols]
                table_data_home.append(row_data)

            # Create a DataFrame for the home team stats
            df_home = pd.DataFrame(table_data_home, columns=['', 'Name', 'Min', 'M', 'A', 'FG%', 'M', 'A', '3P%', 'M', 'A', 'FT%', 'O', 'D', 'T', 'AS', 'PF', 'TO', 'ST', 'BS', 'PTS'])
            
            # Extract the table data fro the away team
            table_data_away = []
            for row in t_awayteam.findAll("tr")[1:]:
                cols = row.findAll("td")
                row_data = [col.text.strip() for col in cols]
                table_data_away.append(row_data)

            # Create a DataFrame for the away team stats
            df_away = pd.DataFrame(table_data_away, columns=['', 'Name', 'Min', 'M', 'A', 'FG%', 'M', 'A', '3P%', 'M', 'A', 'FT%', 'O', 'D', 'T', 'AS', 'PF', 'TO', 'ST', 'BS', 'PTS'])

            # Remove the first empty column
            df_home.drop(df_home.columns[0], axis=1, inplace=True)
            df_away.drop(df_away.columns[0], axis=1, inplace=True)

            # Combine the team stats into one DataFrame
            df_team_stats = pd.concat([df_home, df_away])

            # Create a DataFrame for the team names and scores 
            #df_team = pd.DataFrame({'Team': team_names, 'Total': team_scores})

            # Concatenate the three DataFrames
            #df = pd.concat([df_team, df_table, df_team_stats], axis=1)
            #df.drop(2, inplace=True)
            
          

            df.to_csv(f"/Users/jeanzayas/Desktop/Divergence/Portfolio/Sports Analysis/Basketball/BSN/dataset/{year}/{date_str}_boxscore.csv", index=False)

            df_home.to_csv(f"/Users/jeanzayas/Desktop/Divergence/Portfolio/Sports Analysis/Basketball/BSN/dataset/{year}/{date_str}_home.csv", index=False)
            df_away.to_csv(f"/Users/jeanzayas/Desktop/Divergence/Portfolio/Sports Analysis/Basketball/BSN/dataset/{year}/{date_str}_away.csv", index=False)

           
    except Exception as e:
        print(f"Error: {e}")
        print("There was an error: " + e.args[0] + ". The line where the code failed was " + str(traceback.extract_stack()[-1][1]))


      #exc_type, exc_obj, exc_tb = sys.exc_info()
#        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1] 
#        print(exc_type, fname, exc_tb.tb_lineno)
                

In [307]:
for year in range(2022, 2023):
    for month in range(10, 13):    
        for day in range(1, 100):
            try:
                boxscore_scrape(year,month,day)
                print(f"Successfully scraped boxscore for {year}-{month}-{day}")
                
            except Exception as e:
                print(f"Error: {e}, -{year}-{month}-{day}")
                print("There was an error: " + e.args[0] + ". The line where the code failed was " + str(traceback.extract_stack()[-1][1]))
                exc_type, exc_obj, exc_tb = sys.exc_info()
                fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1] 
                print(exc_type, fname, exc_tb.tb_lineno)
                
          

['4/16/2022', '1st', '2nd', '3rd', '4th', 'TOTAL']
Error: 5 columns passed, passed data had 4 columns
Error: can only concatenate str (not "AssertionError") to str, -2022-10-1
There was an error: can only concatenate str (not "AssertionError") to str. The line where the code failed was 10
<class 'TypeError'> 154958183.py 5
No boxscore found for date 20221002
Successfully scraped boxscore for 2022-10-2
['4/29/2022 8:00:00 PM', '1st', '2nd', '3rd', '4th', 'OT1', 'TOTAL']
Error: 6 columns passed, passed data had 5 columns
Error: can only concatenate str (not "AssertionError") to str, -2022-10-3
There was an error: can only concatenate str (not "AssertionError") to str. The line where the code failed was 10
<class 'TypeError'> 154958183.py 5
['4/30/2022', '1st', '2nd', '3rd', '4th', 'TOTAL']
Error: 5 columns passed, passed data had 4 columns
Error: can only concatenate str (not "AssertionError") to str, -2022-10-4
There was an error: can only concatenate str (not "AssertionError") to str. 

KeyboardInterrupt: 

## Testing on a single cell.. 

I have tested the data acquisitionn of date and gametime ... printed out are the examples of the strip()

In [308]:
import pandas as pd
import datetime

url = 'https://bsnpr.com/juegos/juego?r=BS20201067'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

#tables = soup.findAll("tr")[1]
#tables = soup.findAll("table", class_="tabla_stats")
# Extract the column names from the table header
#headers = [th.text.strip() for th in t_boxscore.findAll("th")[1:]]
# extract the time and date of the game... 
#datetime_str = [th.text.strip() for th in t_boxscore.find("th")][0]
# extract the time and date of the game... 
#datetime_str = [th.text.strip() for th in tables.find("th")][0]
#gametime = datetime_str.split()[1]
#am_pm = datetime_str.split()[-1]
#game_date = datetime_str.split()[0]


#create a datetime object representingg the game start time 
tables = soup.findAll("table", class_="tabla_stats")

t_boxscore = tables[1]

t_hometeam = tables[2]
t_awayteam = tables[3]

# Extract the column names from the table header
headers = [th.text.strip() for th in t_boxscore.findAll("th")]
columns= headers[1:]



            
        
# extract the time and date of the game... 
datetime_str = headers[0]
gametime = datetime_str.split()[1]
am_pm = datetime_str.split()[-1]
game_date = datetime_str.split()[0]
# convert the time to 24 hours format 
game_hour = int(gametime.split(":")[0])
if am_pm == "PM" and game_hour != 12:
    game_hour += 12
game_minute = int(gametime.split(":")[1])
game_second = int(gametime.split(":")[2])
game_time_24hr = datetime.time(game_hour, game_minute, game_second)

df_gameday = pd.DataFrame({'Day': game_date}, index=[0])
df_gametime = pd.DataFrame({'Time': game_time_24hr}, index=[0])
df_ampm = pd.DataFrame({'AM:PM': am_pm}, index=[0])

df_fecha = pd.concat([df_gameday, df_gametime], axis=1)

# Print the column names

print("this is df_gamedate")
df_fecha
#print("this is df_gametime")

#gametime
#game_time_24hr
#datetime_str

this is df_gamedate


,Day,Time
0,11/11/2020,15:00:00


## COPIA PARA INCLUIR EL TIME y date


In [23]:
import requests
import os
from bs4 import BeautifulSoup
import pandas as pd
import datetime
import traceback

def fecha_game(year, month, day):
    try:
        date_str = f"{year}{str(month).rjust(2, '0')}{str(day).rjust(2, '0')}"
        #f"{year}{month}{str(day).rjust(2, '0')}"
        url = 'https://bsnpr.com/juegos/juego?r=BS' + str(date_str)
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')

        tables = soup.findAll("table", class_="tabla_stats")

#tables = soup.findAll("tr")[1]
#tables = soup.findAll("table", class_="tabla_stats")
# Extract the column names from the table header
#headers = [th.text.strip() for th in t_boxscore.findAll("th")[1:]]
# extract the time and date of the game... 
#datetime_str = [th.text.strip() for th in t_boxscore.find("th")][0]
# extract the time and date of the game... 
#datetime_str = [th.text.strip() for th in tables.find("th")][0]
#gametime = datetime_str.split()[1]
#am_pm = datetime_str.split()[-1]
#game_date = datetime_str.split()[0]


        #create a datetime object representingg the game start time 


        t_boxscore = tables[1]

        t_hometeam = tables[2]
        t_awayteam = tables[3]

        # Extract the column names from the table header
        headers = [th.text.strip() for th in t_boxscore.findAll("th")]
        columns= headers[1:]
        # extract the time and date of the game... 
        datetime_str = headers[0]

        game_date = datetime_str.split()[0] 
        gametime = datetime_str.split()[1] if datetime_str.find(" ") > 0 else "8:00"
        am_pm = datetime_str.split()[-1] if datetime_str.find(" ") > 0 else "PM"

        df_fecha = pd.DataFrame({'Day': game_date, 'Time': gametime, 'AM:PM': am_pm}, index=[0])
        # extract the time and date of the game... 
       # datetime_str = headers[0]
       # gametime = datetime_str.split()[1]
       # am_pm = datetime_str.split()[-1]
       # game_date = datetime_str.split()[0]
        # convert the time to 24 hours format 
       # game_hour = int(gametime.split(":")[0])
       # if am_pm == "PM" and game_hour != 12:
        #    game_hour += 12
       # game_minute = int(gametime.split(":")[1])
        #game_second = int(gametime.split(":")[2])
       # game_time_24hr = datetime.time(game_hour, game_minute, game_second)

       # df_gameday = pd.DataFrame({'Day': game_date}, index=[0])
       # df_gametime = pd.DataFrame({'Time': game_time_24hr}, index=[0])
       # df_ampm = pd.DataFrame({'AM:PM': am_pm}, index=[0])

      #  df_fecha = pd.concat([df_gameday, df_gametime], axis=1)
        filepath = f"/Users/jeanzayas/Desktop/Divergence/Portfolio/Sports Analysis/Basketball/BSN/dataset/{year}/{date_str}_fecha.csv"
        df_fecha.to_csv(filepath, index=False)
        #print(df_fecha)
        #print(datetime_str)
        print(df_fecha)
        
        
    except Exception as e:
        print("There was an error: " + e.args[0] + ". The line where the code failed was " + str(traceback.extract_stack()[-1][1]))


In [44]:
for year in range(2009, 2010):
    for month in range(11, 13):    
        for day in range(9, 100):
            try:
                fecha_game(year,month,day)
                print(f"Successfully scraped boxscore for {year}-{month}-{day}")
                
            except Exception as e:
                print(f"Error: {e}, -{year}-{month}-{day}")

        Day  Time AM:PM
0  6/4/2009  8:00    PM
Successfully scraped boxscore for 2009-11-9
        Day  Time AM:PM
0  6/5/2009  8:00    PM
Successfully scraped boxscore for 2009-11-10
        Day  Time AM:PM
0  6/5/2009  8:00    PM
Successfully scraped boxscore for 2009-11-11
        Day  Time AM:PM
0  6/6/2009  8:00    PM
Successfully scraped boxscore for 2009-11-12
        Day  Time AM:PM
0  6/6/2009  8:00    PM
Successfully scraped boxscore for 2009-11-13
        Day  Time AM:PM
0  6/7/2009  8:00    PM
Successfully scraped boxscore for 2009-11-14
        Day  Time AM:PM
0  6/7/2009  8:00    PM
Successfully scraped boxscore for 2009-11-15
        Day  Time AM:PM
0  6/7/2009  8:00    PM
Successfully scraped boxscore for 2009-11-16
        Day  Time AM:PM
0  6/7/2009  8:00    PM
Successfully scraped boxscore for 2009-11-17
        Day  Time AM:PM
0  6/8/2009  8:00    PM
Successfully scraped boxscore for 2009-11-18
        Day  Time AM:PM
0  6/9/2009  8:00    PM
Successfully scraped bo

KeyboardInterrupt: 

In [245]:
import datetime
import requests
from bs4 import BeautifulSoup
import pandas as pd


def
    try:
        date_str = f"{year}{str(month).rjust(2, '0')}{str(day).rjust(2, '0')}"
        #f"{year}{month}{str(day).rjust(2, '0')}"
        url = 'https://bsnpr.com/juegos/juego?r=BS' + str(date_str)
        response = requests.get(url)
        
        tables = soup.findAll("table", class_="tabla_stats")
        t_boxscore = tables[1]
        t_date = [th.text.strip()for th in t_boxscore.findAll("th")]
        t_hometeam = tables[2]
        t_awayteam = tables[3]

        # Extract the column names from the table header
        headers = [th.text.strip() for th in t_boxscore.findAll("th")][1:]


        # extract the time and date of the game... 
        datetime_str = t_date[0]

        #split the datetime string into its components 
        #date_str = datetime_str.split()[0]
        #time_str = datetime_str.split()[1]

        game_day_date = []
        time_start_game = []
        for t_boxscore in tables:
            # extract the time and date of the game... 
            datetime_str = t_date[0]
            gametime = datetime_str.split()[1]
            am_pm = datetime_str.split()[-1]
            game_date = datetime_str.split()[0]

            # convert the time to 24 hours format 
            game_hour = int(gametime.split(":")[0])
            if am_pm == "PM" and game_hour != 12:
                game_hour += 12
            game_minute = int(gametime.split(":")[1])
            game_second = int(gametime.split(":")[2])


            # create datetime object for game start time
            #game_start_time = datetime.datetime.strptime(game_date + ' ' + str(game_hour) + ':' + str(game_minute) + ':' + str(game_second), '%Y-%m-%d %H:%M:%S')
            #game_start_times.append(game_start_time)


        # extract the team names and scores from the table_boxscore
        team_names = []
        team_scores = []
        for row in t_boxscore.findAll("tr")[1:]:
            cols = row.findAll("td")
            team_names.append(cols[0].text.strip())
            team_scores.append(cols[-1].text.strip())

        # Extract the table data
        table_data = []
        for row in t_boxscore.findAll("tr")[1:]:
            cols = row.findAll("td")[1:-1]
            row_data = [col.text.strip() for col in cols]
            table_data.append(row_data)

        df_gameday = pd.DataFrame({'Day': game_date}, index=[0])
        df_gametime = pd.DataFrame({'Time': game_time_24hr}, index=[0])

        df_fecha = pd.concat([df_gameday, df_gametime], axis=1)

        # Concatenate the two DataFrames
        df = pd.concat([df_team, pd.DataFrame(table_data[:4], columns=headers)], axis=1)
        # Extract the team names and scores from the DataFrame

        # Create a DataFrame for the team names and scores 
        df_team = pd.DataFrame({'Team': team_names})
          # Create a DataFrame for the team names and scores 


        # Concatenate the two DataFrames
        df = pd.concat([df_team, df_table], axis=1)
        df.drop(2, inplace=True)

        df= pd.concat([df, df_fecha], axis=1)
        # Extract the team names and scores from the DataFrame

        home_team = df.iloc[0]['Team']
        home_score = df.iloc[0]['Total']
        away_team = df.iloc[1]['Team']
        away_score = df.iloc[1]['Total']
        # Print the final score in the
        print(f"{home_team} {home_score} - {away_team} {away_score}")

IndexError: list index out of range

In [ ]:
for year in range(2023, 2024):
    for month in range(10, 13):
        for day in range(1, 100):
            try:
                boxscore = get_team_boxscore(year, month, day)
                if boxscore is not None:
                    print(f"Successfully scraped boxscore for {year}-{month}-{day}")
                else:
                    print(f"No boxscore available for {year}-{month}-{day}")
            except Exception as e:
                print(f"Error: {e} - {year}-{month}-{day}")
